In [3]:
import torch.nn as nn
import numpy as np
import nltk
import torch
from nltk.stem.porter import PorterStemmer
from torch.nn import functional as F
from torch.utils.data import TensorDataset


#NeuralNet

In [4]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.key = nn.Linear(input_size, hidden_size, bias=False)
        self.query = nn.Linear(input_size, hidden_size, bias=False)
        self.value = nn.Linear(input_size, hidden_size, bias=False)
        self.relu = nn.ReLU()

    def forward(self, x):
        key = self.key(x)   
        query = self.query(x) 
        k = key.transpose(-2, -1)  
        wei = query @ k 
        
        # compute attention scores ("affinities")
        tril = torch.tril(torch.ones_like(wei))
        wei = wei.masked_fill(tril == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        
        # perform the weighted aggregation of the values
        v = self.value(x) 
        out = wei @ v 
        return out


In [5]:


def bag_of_words(tokenized_sentence,words):
    sentence_word=[PorterStemmer().stem(word.lower()) for word in tokenized_sentence]
    bag=np.zeros(len(words),dtype=np.float32)

    for idx , w in enumerate(words):
        if w in sentence_word:
            bag[idx]=1

    return bag

#Training Data

In [6]:
import numpy as np
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

with open('data.json','r') as f:
    data=json.load(f)

all_words=[]
keys=[]
xy=[]
x_train=[]
y_train=[]


for data in data['data']:
    key=data['key']
    keys.append(key)

    for query in data['query']:
        print(query)
        w=nltk.word_tokenize(query)
        all_words.extend(w)
        xy.append((w,key))

ignore_words=[',','?','/','.','!']
all_words=[PorterStemmer().stem(w.lower()) for w in all_words if w not in ignore_words]
all_words=sorted(set(all_words))
keys=sorted(set(keys))

for (query_sentence,key) in xy:
    bag=bag_of_words(query_sentence,all_words)
    x_train.append(bag)

    label=keys.index(key)
    y_train.append(label)

x_train=np.array(x_train)
y_train=np.array(y_train)


num_epochs=1000
batch_size=15
learning_rate=0.001
input_size=len(x_train[0])
hidden_size=20
output_size=len(keys)

class dataset(Dataset):

    def __init__(self):
        self.n_samples=len(x_train)
        self.x_data=x_train
        self.y_data=y_train

    def __getitem__(self,index):
        return self.x_data[index],self.y_data[index]
    
    def __len__(self):
        return self.n_samples
    
dataset=TensorDataset(torch.tensor(x_train), torch.tensor(y_train))
batch_size=2
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)

embedding_dim=50
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Head(input_size,hidden_size,output_size).to(device)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)


for epoch in range(num_epochs):
    for (words,labels) in train_loader:
        words=words.to(device)
        labels=labels.to(dtype=torch.long).to(device)
        outputs=model(words)
        loss=criterion(outputs,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

print(f'Final loss : {loss.item():.4f}')



hello assistant
hii nassistant
hey
wake up
how are you
hi
hello
bye
goo
good bye
quit
see you later
i am going
who are you
what is your name
what is your purpose
your name please
who made you
what is time
time kitne hua
current time
could you please tell me the time
time in system
what is date
date kitne hua
current date
could you please tell me the date
date in system
today's date
what is day
day kitne hua
current day
could you please tell me the day
day in system
today's day
lock the device
can you lock my system
lock pc
lock karo
what is the temperature
could you please tell me the temperature
what's the temperature outside
temperature please
what is the weather condition
how is the the weather
weather condition please
is weather cool
is weather hot
how is the weather outside
open google
please open the google
google front page
google open page
open youtube
please open the youtube
youtube front page
youtube open page
open word
please open the word
open word document
word document pl

In [7]:
data={
    "model_state":model.state_dict(),"input_size":input_size,"hidden_size":hidden_size,
    "output_size":output_size,"all_words":all_words,"keys":keys
}

torch.save(data,"output.pth")


In [8]:
#required packages

import random
import json
import torch
import pyttsx3
import speech_recognition as sr
import datetime
from datetime import datetime
from googlesearch import search
import wikipedia
import pywhatkit
import webbrowser
import os
import re
import sys

#TakeCmd
def take_cmd():
    r=sr.Recognizer()
    with sr.Microphone() as source:
        print('listening...')
        r.pause_threshold=1
        audio=r.listen(source)
    try:
        query=r.recognize_google(audio,language='en-in')
          
    except Exception as e:
        print('please say that again...')
        return 'None'
    query=str(query)
    return query.lower()

In [9]:
#RespondCmd
def respond(cmd):
    engine=pyttsx3.init()
    voices=engine.getProperty('voices')
    #print(voices[1])
    engine.setProperty('voice',voices[1].id)

    engine.say(cmd)
    engine.runAndWait()

def wish():
    respond('hi..')
    respond('i can guide you for anything...')


In [10]:
import ctypes
import requests
from bs4 import BeautifulSoup
import requests


# Define the LockWorkStation function from the Win32 API
def lock():
    ctypes.windll.user32.LockWorkStation()

#--------------------------

def get_temperature(place):
    url = f"https://www.google.com/search?q={place}+temperature"

    try:
        response = requests.get(url)
        data = BeautifulSoup(response.text, 'html.parser')
        temp = data.find("div", class_="BNeawe").text
        
        return temp

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None
    
#-------------------------

def google_search(query):
    url = f"https://www.google.com/search?q={query}"
    
    webbrowser.open(url)

#-------------------------

def get_current_city():
    response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?latlng=latitude,longitude&key=YOUR_API_KEY')
    data = response.json()
    
    if response.status_code == 200 and data['status'] == 'OK':
        results = data['results']
        
        if results:
            address_components = results[0]['address_components']
            
            for component in address_components:
                if 'locality' in component['types']:
                    city = component['long_name']
                    respond(f"Current City is {city}")
                    return
        
        respond("City information not available.")
    else:
        respond("Failed to retrieve current location.")

#---------------------------

def Date():
    current_date = datetime.date.today()
    print("Today's date:", current_date)
    respond(current_date)

#----------------------------

def Day():
    current_date = datetime.date.today()

    day_of_week = current_date.weekday()
    weekday_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    weekday_name = weekday_names[day_of_week]
    respond("its..")
    respond(weekday_name)


In [18]:

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')


with open('data.json','r') as json_data:
    data_json=json.load(json_data)
    
data = torch.load('output.pth')
input_size, hidden_size, output_size = data['input_size'], data['hidden_size'], data['output_size']
all_words, keys, model_state = data['all_words'], data['keys'], data['model_state']

model=Head(input_size,hidden_size,output_size).to(device)
model.load_state_dict(model_state)
model.eval()

if __name__=='__main__':
    wish()
    while True:
    
        speech=take_cmd()

        speech=nltk.word_tokenize(speech)
        comb_speech=" ".join(speech)
        X=bag_of_words(speech,all_words)
        X=X.reshape(1,X.shape[0])
        X=torch.from_numpy(X).to(device)
        print("X")
        print(X)
        output=model(X)
        _,predicted = torch.max(output,dim=1)
        print("_:",_)
        print("predicted:",predicted)
        print("keys:",keys)
        key=keys[predicted.item()]
        print("1st key is",key)
        probs=torch.softmax(output,dim=1)
        prob=probs[0][predicted.item()]

        if prob.item() > 0.70:
            for data in data_json['data']:
                print("data is",data)
                if key == data["key"]:
                    print("2nd key is",key)
                    reply=random.choice(data["value"])
                    if "time" in reply:
                        dt = datetime.now().time()
                        # convert to 12 hour time format
                        formatted_time = dt.strftime("%I:%M %p")
                        respond(formatted_time)
                        print(formatted_time) 

                    elif "date" in reply:
                        current_date = datetime.date.today()
                        print("Today's date:", current_date)
                        respond(current_date)
                    
                    elif 'day' in reply:
                        Day()
                    
                    elif "lock" in reply:
                        respond("locking the device")
                        lock()
                    
                    elif "temperature" in reply or "weather" in reply:

                        replacements = {
                            "what":"","in":"","is":"","the":"",
                            "what is weather": "",
                            "what is temperature": "",
                            "how is the":"",
                            "temperature":"",
                            "weather":"",
                            "cool":"",
                            "hot":"",
                        }
                        words = reply.split()
                        replaced_words = [replacements.get(word, word) for word in words]
                        place = " ".join(replaced_words)
                        temp=get_temperature(place)
                        respond(f"The temperature in {place} is {temp}")
                        temp = ''.join(filter(str.isdigit, temp))
                        temp=int(temp)
                        if 0<temp<15:
                            respond("the weather is very cool")
                        elif 15<temp<35:
                            respond("the weather is moderate")
                        elif temp>35:
                            respond("the weather is hott")


                    elif "google" in key:
                        webbrowser.open("google.com")

                    
                    elif 'open youtube' in reply:
                        webbrowser.open('youtube.com')

                    elif 'what' in reply or 'search' in reply:
                        google_search(comb_speech)
                        
                    elif "word" in reply:
                        os.startfile("C:/ProgramData/Microsoft/Windows/Start Menu/Programs/Word.lnk")

                    elif 'excel' in reply:
                        os.startfile("C:/ProgramData/Microsoft/Windows/Start Menu/Programs/Excel.lnk")

                    elif 'vs code' in reply:
                        os.startfile("C:/Users/ASUS/AppData/Roaming/Microsoft/Windows/Start Menu/Programs/Visual Studio Code")
    
                    elif 'git' in reply:
                        webbrowser.open('https://github.com/talabathulamohankrishna')

                    elif 'add' in reply or 'plus' in reply or 'sum' in reply:
                        numbers = re.findall(r'\d+', comb_speech)
                        numbers = [int(num) for num in numbers]
                        num=0
                        for i in numbers:
                            num=i+num
                        respond(num)
                
                    elif 'subtract' in reply or 'minus' in reply or 'remove' in reply:
                        numbers = re.findall(r'\d+', comb_speech)
                        numbers = [int(num) for num in numbers]

                        if len(numbers) >= 2:
                            sub = numbers[1] - numbers[0]
                            respond(sub)
                        else:
                            respond("Not enough numbers provided for subtraction.")


                    elif "location" in reply or "city" in reply:
                        get_current_city()


                    elif "bye" in reply or "see you soon" in reply or "i am going" in reply:
                        respond(reply)
                        sys.exit()
                    
                    else:
                        respond(reply)
        else:
            respond("sorry...")
            respond("i have no related data in my database")
            print("sorry...I have no related data in my database")

        print(comb_speech)


listening...
please say that again...
X
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.]])
_: tensor([0.], grad_fn=<MaxBackward0>)
predicted: tensor([0])
keys: ['addition', 'bye', 'date', 'day', 'excel', 'git', 'google', 'location', 'lock', 'name', 'search', 'subtraction', 'temperature', 'time', 'vs code', 'wish', 'word', 'youtube']
1st key is addition
sorry...I have no related data in my database
listening...
X
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0

KeyboardInterrupt: 

: 

In [ ]:

import datetime

current_date = datetime.date.today()
print("Today's date:", current_date)
respond(current_date)

Today's date: 2023-05-31


In [15]:
import re

comb_speech = "subtract 2 from 1"
numbers = re.findall(r'\d+', comb_speech)
numbers = [int(num) for num in numbers]

if len(numbers) >= 2:
    sub = numbers[1] - numbers[0]
    respond(sub)
else:
    respond("Not enough numbers provided for subtraction.")
